Introduction

# Exemple : Concaténation et nettoyage des données OPM

Ce notebook illustre deux étapes clés du pipeline :
1. **Concaténation** de deux tables (2015 et 2024)
2. **Nettoyage** d'une table de niveaux d'âge (`DTagelvl_Clean`)

Les données proviennent de l'Office of Personnel Management (OPM, USA) et sont utilisées pour l'analyse de l'attrition.


Imports

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from pandas.api.types import CategoricalDtype


Concaténation des tables

In [ ]:
# Définir les chemins relatifs
data_dir = Path("../data/raw")

# Charger les fichiers
df_1 = pd.read_csv(data_dir / "DTagy_2015.txt")
df_2 = pd.read_csv(data_dir / "DTagy_2024.txt")

# Concaténer les deux DataFrames
df_concat = pd.concat([df_1, df_2], ignore_index=True)

# Supprimer les doublons
df_concat.drop_duplicates(inplace=True)

# Exporter vers CSV dans data/data/
output_path = Path("../data/data/DTagy_Clean.csv")
df_concat.to_csv(output_path, index=False)

print(f"Fichier concaténé sauvegardé : {output_path}")


Nettoyage de la table - DTagelvl_Clean


In [ ]:
# Charger le fichier nettoyé
df_0 = pd.read_csv(Path("../data/data/DTagelvl_Clean.csv"))

# Supprimer une ligne spécifique
df_0.drop([11], inplace=True)

# Modifier certaines valeurs
df_0.replace(
    {"Less than 20": "<20",
     "65 or more": ">=65"}, 
    inplace=True
)

# Définir l'ordre des catégories d'âge
age_order = CategoricalDtype(
    categories=["<20", "20-24", "25-29", "30-34", "35-39", "40-44", 
                "45-49", "50-54", "55-59", "60-64", ">=65"],
    ordered=True
)

df_0["AGELVLT"] = df_0["AGELVLT"].astype(age_order)

# Extraire la borne supérieure de l'âge
def extract_upper_bound(age_category):
    if age_category == "Unspecified":
        return np.nan 
    elif age_category == "<20":
        return 20 
    elif age_category == ">=65":
        return 65  
    else:
        return int(age_category.split('-')[-1]) 

df_0["AGELVLT_Age"] = df_0["AGELVLT"].apply(extract_upper_bound).astype("Int64")

# Renommer la table finale
DTagelvl_net = df_0

# Exporter vers CSV
DTagelvl_net.to_csv(Path("../data/data/DTagelvl_net.csv"), index=False)

print("Table nettoyée exportée.")


## Conclusion

Ce notebook démontre :
- La fusion de deux tables brutes en un dataset unique
- Le nettoyage et la transformation d'une table de niveaux d'âge
- L'export des fichiers prêts pour analyse

Ces étapes font partie du pipeline complet d'analyse de l'attrition.
